# Modeling the evolution of an etch recipe in Deep silicon etching

To Do for next version:
Cellular automata implementation of dry silicon etching.
Cells in a 3D grid are intialized with a state value of 1. As etching occures, the cell state is subtracted from at a user-defined rate that, for isotropic steps, its adjusted by the calculated angle between the surface normals and the center vertical axis of the etched feature.

Simplify Etch conditions and simulation parameters

Verify recipe steps for steps that are  Bosch or isotropic etching, vert_rate, horiz_rate, and bosch_vert_step. 

add image of a mask superimposed to the simulation

During this notebook you will learn to  predict and model etch profiles from dry silicon etching based on known etch rates. You will be able to change multiple parameters, including the etch rate and exposure time.

To model continuous etching into silicon wafer on a Deep Silicon etching tool you will need to specify the correct parameters. In this case, you will simulate dry etching in a gas plasma.

In [5]:
    k_b = 1.38e-23  # Boltzman constant [J/K]
    T_s = 100 + 274.15  # substrate temperature [K]
    k_0 = (0,30,30)
    F_r = 150  # flow rate of SF6 [sccm]
    bias = -150 #volts

Now, precribing custom Bosch, isotropic, and tapered (combined bosch and isotropic) etching steps can be modifyied if necessary in the following:

In [ ]:
for i_bosch in range(recipe_steps[step]['bosch']):
                    if len(str(i_bosch)) < 3:
                        if len(str(i_bosch)) == 1: 
                            if i_bosch == 9:
                                i_bosch_str = '0' + str(i_bosch+1)
                            else:
                                i_bosch_str = '00' + str(i_bosch+1)
                        elif len(str(i_bosch)) == 2: 
                            if i_bosch == 99:
                                i_bosch_str = str(i_bosch+1)
                            else:
                                i_bosch_str = '0' + str(i_bosch+1)
                                
                    # initial bosch cycle key
                    key = step + '_bosch-iso' + i_cycle_str + \
                          '_bosch' + i_bosch_str + '_isotime0'
                    etch_grid[key] = []

The surface is evolved by using the computed normals of the surface and stepping points back along the normals by some user defined vertical and horizontal etch rates.


<img align="center" src="img/DESi.png" width="750" />
<h3 align="center">From the class, remember the effect of Deep dry etching of poly-Si</h3>


In [ ]:
    
C4F8 = 100  # sccm cubic centimeters per minute  
SF6 = 300  # sccm
bias = 10  # volts
time = 600  # seconds
opening = 100  # um

plt.close('all')

# load mask
im_dir = '/ExampleMasks/'
im_file = 'fillet_square.png'
im_path = im_dir + im_file
curr_im = cv2.imread(im_path, cv2.IMREAD_ANYDEPTH)   
curr_im = cv2.GaussianBlur(curr_im,(3,3),0)


rgb_im = cv2.cvtColor(curr_im, cv2.COLOR_GRAY2RGB)
     
cont_im, conts, hier = cv2.findContours(curr_im, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)    
conts_im = cv2.drawContours(rgb_im, conts, -1, (0,255,0),3)

t_start = 0
t_end = 600  # seconds
t_step = 5
rot90_mat = np.array([[np.cos(np.pi/2), -np.sin(np.pi/2)],
                      [np.sin(np.pi/2), np.cos(np.pi/2)]])
vert_rate = 287/600  # um/s

horiz_rate = 77/600  # um/s
pixel_um_conv = 251/90.4672  # px/um
cmap = 'gnuplot'  # 'inferno' 'viridis'  # 'hot'
vmin = -290  # expected range of depth for color bar (min)
vmax = 0

Now try changing the parameters to approximate the etching results as follows:


<img align="center" src="img/DEtop.gif" width="750" />
<h3 align="center">Top view of Dry Etching with the fillet mask example. Time steps of 5s.</h3>


<img src="img/DEiso.gif" width="750" />
<h3 align="center">Isometric view of Dry Etching with the fillet mask example. Time steps of 5s.</h3>


From the class, remember the equipment diagram used for Deep dry etching.
<img src="img/DEeqp.png" width="500" />


